In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

#We may want these at some point for transforming our output:
#from scipy.special import logit, expit

In [3]:
%run ../data/features-grouped.ipynb
%run ../data/model-information.ipynb

In [7]:
filepath = '../data/data-reduced-train.csv'
data = pd.read_csv(filepath)

In [27]:
data_train, data_val = train_test_split(data,
                                        test_size=0.2,
                                        random_state=42)

In [8]:
#n_neighbors = 10 #number of neighbors for kNN imputation
features = data.columns[4:]
target = '% Adults with Diabetes'

In [28]:
xgb_pipe = Pipeline([('impute', KNNImputer()),
                     ('xgb', XGBRegressor())])

param_grid = {
    'impute__n_neighbors': [10, 15],
    'xgb__n_estimators': [50, 100, 150],
    'xgb__max_depth': [3, 5, 7],
    'xgb__learning_rate': [0.1, 0.15],
    'xgb__reg_alpha': [0, 0.1, 1],
    'xgb__reg_lambda': [0, 1, 5]
}

xgb_grid = GridSearchCV(xgb_pipe,
                        param_grid=param_grid,
                        scoring='neg_root_mean_squared_error',
                        cv=3)

In [29]:
xgb_grid.fit(data_train[features], data_train[target])

/Users/nealedgren/opt/anaconda3/envs/erdos_fall_2024/lib/python3.12/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('impute', KNNImputer()),
                                       ('xgb',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     callbacks=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     device=None,
                                                     early_stopping_rounds=None,
                                                     enable_categorical=False,
                                                     eval_metric=None,
                                                     feature_types=None,
                                                     gamma=None,
                                                     grow_policy=None,
                                                     importance_type=None,
                                                     in...
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=None, ...))]),
             param_grid={'impute__n_neighbors': [10, 15],
                         'xgb__learning_rate': [0.1, 0.15],
                         'xgb__max_depth': [3, 5, 7],
                         'xgb__n_estimators': [50, 100, 150],
                         'xgb__reg_alpha': [0, 0.1, 1],
                         'xgb__reg_lambda': [0, 1, 5]},
             scoring='neg_root_mean_squared_error')

In [34]:
xgb_grid.best_params_

{'impute__n_neighbors': 10,
 'xgb__learning_rate': 0.15,
 'xgb__max_depth': 3,
 'xgb__n_estimators': 150,
 'xgb__reg_alpha': 0,
 'xgb__reg_lambda': 5}

In [35]:
xgb_grid.best_score_

np.float64(-0.522992124902723)

In [36]:
xgb_grid.best_estimator_

Pipeline(steps=[('impute', KNNImputer(n_neighbors=10)),
                ('xgb',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.15,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=3, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=150, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [37]:
model = xgb_grid.best_estimator_

model.fit(data_train[features], data_train[target])

training_preds = model.predict(data_train[features])
root_mean_squared_error(data_train[target], training_preds)

np.float64(0.28639533372919745)

In [38]:
val_preds = model.predict(data_val[features])
root_mean_squared_error(data_val[target], val_preds)

np.float64(0.5201252506177477)

In [ ]:
params = {
    'n_estimators': 150,
    'max_depth': 5,
    'learning_rate': 0.1,
    'reg_alpha': 1,
    'reg_lambda': 1
}

In [24]:
xgb_pipe_new = xgb_pipe = Pipeline([('impute', KNNImputer(n_neighbors=15)),
                     ('xgb', XGBRegressor(**params))])

xgb_pipe_new.fit(data[features], data[target])

Pipeline(steps=[('impute', KNNImputer(n_neighbors=15)),
                ('xgb',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=5, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=150, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [ ]:
preds = xgb_pipe_new.predict(data[features])
root_mean_squared_error(data[target], preds)

np.float64(0.1670568370947467)

In [39]:
linear_pipe = Pipeline([('impute', KNNImputer(n_neighbors=10)),
                        ('scale', StandardScaler()), #For coefficient size comparison
                        ('linreg', LinearRegression())])

linear_pipe.fit(data_train[features], data_train[target])


Pipeline(steps=[('impute', KNNImputer(n_neighbors=10)),
                ('scale', StandardScaler()), ('linreg', LinearRegression())])

In [43]:
linear_training_preds = linear_pipe.predict(data_train[features])
print(root_mean_squared_error(data_train[target], linear_training_preds))

linear_val_preds = linear_pipe.predict(data_val[features])
print(root_mean_squared_error(data_val[target], linear_val_preds))

0.472038732165765
0.4969364285883717


In [ ]:
forest_pipe = Pipeline([('impute', KNNImputer()),
                        ('rfr', RandomForestRegressor())])